# Getting data ready for training

In [ ]:
from scripts.data_preparation import get_data_ready_for_training

X_train, X_test, y_train, y_test, multi_label_binarizer = get_data_ready_for_training(
    sampling_rate=100, dataset_path="../ecg_dataset", need_3D_input=True
)

# Splitting training into training|validation

In [1]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

# Model selection

In [ ]:
# from networks.cnn_v2 import Cnn
#
# model = Cnn()

from tensorflow import keras

model = keras.models.Sequential(
    [
        keras.layers.Conv2D(
            64, 7, activation="relu", padding="same", input_shape=[1000, 12, 1]
        ),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(5, activation="sigmoid"),
    ]
)


# Callbacks

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

callbacks = [
    EarlyStopping(monitor="val_AUC", mode="max", verbose=1, patience=10),
    ReduceLROnPlateau(
        monitor="val_AUC",
        factor=0.1,
        patience=5,
        verbose=1,
        mode="max",
        min_delta=0.0001,
        cooldown=0,
        min_lr=0,
    ),
]

# Model compilation

In [ ]:
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy, Recall, Precision, AUC

model.compile(
    loss=BinaryCrossentropy(),
    optimizer=Adam(learning_rate=0.001),
    metrics=[
        BinaryAccuracy(name="accuracy", dtype=None, threshold=0.5),
        Recall(name="Recall"),
        Precision(name="Precision"),
        AUC(
            num_thresholds=200,
            curve="ROC",
            summation_method="interpolation",
            name="AUC",
            dtype=None,
            thresholds=None,
            multi_label=True,
            label_weights=None,
        ),
    ],
)

# Model architecture

In [ ]:
# model.model().summary()
model.summary()

# Model training

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=8,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    shuffle=True,
)

# Saving model

In [ ]:
model.save(filepath="../saved_models/cnn_v3", save_format="tf")

# Model evaluation

In [ ]:
model.evaluate(X_test, y_test)

In [1]:
import numpy as np

threshold = 0.5
y_pred = (model.predict(X_test) > threshold) * 1

In [ ]:
from scripts.data_evaluation import compute_accuracy

compute_accuracy(y_test, y_pred)

In [1]:
from scripts.data_evaluation import plot_confusion_matrix

plot_confusion_matrix(y_test, y_pred, multi_label_binarizer.classes_)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

# Show predictions on sample patient

In [ ]:
class_names = np.array(multi_label_binarizer.classes_)

patient_ecg = np.expand_dims(X_test[0], axis=0)
patient_ecg_prob = (model.predict(patient_ecg) > 0.5) * 1
sorted_categories = np.argsort(patient_ecg_prob[0])[:-6:-1]

for i in range(5):
    print(
        f"{class_names[sorted_categories[i]]}: {patient_ecg_prob[0][sorted_categories[i]]}"
    )

In [ ]:
y_test[0]

In [ ]:
class_names